# Bayesian Optimization with Ax Tutorial

This tutorial demonstrates how to use Bayesian Optimization with the Ax library to optimize machine learning model parameters and other experiments efficiently. Ax is a powerful tool developed by Facebook for automating experimentation and optimization.

## What is Bayesian Optimization?

Bayesian Optimization is an efficient strategy for optimizing black-box functions that are expensive to evaluate. It's particularly well-suited for high-dimensional spaces and situations where sampling data is costly.

## Introduction to Ax

Ax is an accessible, modular, and efficient library that supports both gradient-based optimization and Bayesian Optimization. It is designed to automate the process of optimizing complex experiments, like tuning hyperparameters for machine learning models.


# Basic Ax Optimization

## Setting Up the Environment

After installing Ax, we need to import the necessary libraries to set up our optimization problem.


In [9]:
# Import Ax libraries
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting


## Defining the Optimization Problem

For this tutorial, we'll optimize a simple synthetic function as our objective. Our goal is to find the minimum value of this function.


In [10]:
# Define the objective function
def objective_function(parameters):
    x = parameters.get("x")
    y = parameters.get("y")
    objective_value = (x - 0.5)**2 + (y - 0.5)**2  # Simple convex function
    return {"objective_value": (objective_value, 0.0)}


## Configuring Bayesian Optimization in Ax

We will now configure the Bayesian Optimization process in Ax by setting up an experiment. The evaluation function calculates an objective value based on the parameters x and y. We are using a simple quadratic function for this problem. This function matches our originally defined objective function. The optimize function takes a few inputs. The first is a list of parameters to be optimized. Each is given a name, type, and optimization bounds. Next, you have to define an evaluation function for the model to use to evaluate the generated parameters. Next is the objective to be optimized. This is the name of the value. This needs to match the name defined in the objective function. Next, tell the model if it's trying to minimize or maximize. Lastly you can specify the number of trials it needs to run. The higher this number is the more accurate it will be. 


In [11]:
# Correct way to initialize AxClient for Bayesian Optimization with objective setup
from ax.service.managed_loop import optimize

def evaluation_function(parameterization):
    x = parameterization.get("x")
    y = parameterization.get("y")
    return (x - 0.5)**2 + (y - 0.5)**2  # Example objective function

best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "x", "type": "range", "bounds": [0.0, 1.0]},
        {"name": "y", "type": "range", "bounds": [0.0, 1.0]},
    ],
    evaluation_function=evaluation_function,
    objective_name='objective_value',
    minimize=True,  # Specifies that we are minimizing our objective
    total_trials=15
)

print("Best Parameters:", best_parameters)





[INFO 09-25 19:52:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 19:52:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter y. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 19:52:16] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='y', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 09-25 19:52:17] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there is at least one ordered parameter and there are no unordered categorical parameters.
[INFO 09-25 19:52:17] ax.modelbridge.dispatch_utils: Calculating th

Best Parameters: {'x': 0.4824572684758933, 'y': 0.4893518126070221}


## Using Ax to Optimize Within a Dataset

First we need to import our libraries. The biggest new one that we need is the euclidean_distances. This is useful when trying to optimize from values within a dataset. It gives the model an 'evaluation function' to use. 

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from ax.service.managed_loop import optimize

Next we need to import and clean up our data. It's important to do this step as it can cause the Ax model to run into errors down the line or be unable to efficiently find the optimal value. Let's inspect our data and see what we're working with. 

In [13]:
# Load the dataset
data = pd.read_csv('AgNP_dataset.csv')

# Drop or fill NaN values
data = data.dropna()  # Option 1: Drop rows with NaN values
# or
data = data.fillna(data.mean())  # Option 2: Fill NaN values with column mean

# Define the inputs (X) and the output (y)
y = data['loss']
X = data.drop('loss', axis=1)

data.describe()

,QAgNO3(%),Qpva(%),Qtsc(%),Qseed(%),Qtot(uL/min),loss
count,3295.000000,3295.000000,3295.000000,3295.000000,3295.000000,3295.000000
mean,23.904572,24.408302,8.781410,6.810531,706.117347,0.502575
std,8.492450,8.571230,7.492459,4.503125,207.889927,0.198356
min,4.530000,9.999518,0.500000,0.498852,200.000000,0.131345
25%,18.529201,17.880991,3.999102,4.000384,600.000000,0.321620
50%,25.099136,22.819075,5.848561,6.500000,807.580000,0.491461
75%,30.500000,32.399194,12.000000,8.660701,815.000000,0.661212
max,42.809816,40.001015,30.500000,19.500000,983.000000,0.910637


3295 is a great amount of data to be working with. We can see that our loss varies bewteen .131 and .910

Next we need to create our model and test it. The optimize function uses very similar inputs as the model above, except this time it has a lot more parameters to optimize than before. Since we are optimizing within a dataset we can set the bounds to be the max and min of each input column. 

In [14]:
def evaluation_function(parameterization):
    # Convert parameterization to a DataFrame
    param_df = pd.DataFrame([parameterization])
    
    # Compute the distance between the parameterization and all rows in the test set
    distances = euclidean_distances(param_df, X)
    
    # Find the index of the closest row
    closest_index = np.argmin(distances)
    
    # Get the corresponding output value
    output_value = y.iloc[closest_index]
    
    # Return the output value as the objective
    return {"objective_value": output_value}

# Define the parameters to optimize
parameters = [
    {"name": "QAgNO3(%)", "type": "range", "bounds": [float(X['QAgNO3(%)'].min()), float(X['QAgNO3(%)'].max())]},
    {"name": "Qpva(%)", "type": "range", "bounds": [float(X['Qpva(%)'].min()), float(X['Qpva(%)'].max())]},
    {"name": "Qtsc(%)", "type": "range", "bounds": [float(X['Qtsc(%)'].min()), float(X['Qtsc(%)'].max())]},
    {"name": "Qseed(%)", "type": "range", "bounds": [float(X['Qseed(%)'].min()), float(X['Qseed(%)'].max())]},
    {"name": "Qtot(uL/min)", "type": "range", "bounds": [float(X['Qtot(uL/min)'].min()), float(X['Qtot(uL/min)'].max())]}
]

# Run the optimization
best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=evaluation_function,
    objective_name='objective_value',
    minimize=True,
    total_trials=100
)

[INFO 09-25 19:52:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter QAgNO3(%). If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 19:52:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Qpva(%). If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 19:52:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Qtsc(%). If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-25 19:52:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Qseed(%). If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in p

Finally, let's see what our best parameters are. Keep in mind this can be influenced by the number of trials that the model is running. If it has a very low amount it may not have the chance to find the optimal parameters. 

In [15]:
print("Best Parameters:", best_parameters)
print("Best Objective Value:", values[0])

Best Parameters: {'QAgNO3(%)': 36.95316824582501, 'Qpva(%)': 14.498258205867465, 'Qtsc(%)': 13.72584530183296, 'Qseed(%)': 0.498851653, 'Qtot(uL/min)': 887.5740031672151}
Best Objective Value: {'objective_value': 0.15935830981454724}


## Hyperparameter Optimization with Ax

In this section we will train a random forest regressor on predicting the loss value from the input data that we used before. We will then use an Ax model to tune the hyperparameters of this model.

First we will import the libraries required. We will be using a random forest model from sklearn

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ax.service.managed_loop import optimize
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Next we will load the same dataset. Everything here is the same as before except we are going to split it into train/test splits in order to be able to correctly score our random forest regression model. It's important to specify the random_state in order to have reproducability between runs.

In [17]:
# Load the dataset
data = pd.read_csv('AgNP_dataset.csv')


# Check for NaN values
print(data.isna().sum())

# Drop or fill NaN values
data = data.dropna()  # Option 1: Drop rows with NaN values
# or
data = data.fillna(data.mean())  # Option 2: Fill NaN values with column mean

# Define the inputs (X) and the output (y)
y = data['loss']
X = data.drop('loss', axis=1)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

QAgNO3(%)       0
Qpva(%)         0
Qtsc(%)         0
Qseed(%)        0
Qtot(uL/min)    0
loss            0
dtype: int64


Next we will train and score our model. The details are covered in the random_forest notebook.

In [18]:
# Train a RandomForestRegressor model
model = RandomForestRegressor(random_state=42, n_estimators=10, max_depth=5, min_samples_split=5, min_samples_leaf=2)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.0038889048983534588


We have a pretty good MSE to start! However, we can do better than it. We will now use an Ax model to tune the parameters of this random forest model. Much like the optimization above, we list out the parameters we want to tune. however, our evaluation function trains and fits the random forest model with the found parameters, then uses the MSE score to find the optimal values. 

In [19]:
def evaluation_function(parameterization):
    # Extract hyperparameters from parameterization
    n_estimators = parameterization.get("n_estimators")
    max_depth = parameterization.get("max_depth")
    min_samples_split = parameterization.get("min_samples_split")
    min_samples_leaf = parameterization.get("min_samples_leaf")
    
    # Train the model with given hyperparameters
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Predict and calculate MSE on the test set
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    
    # Return the MSE as the objective value to minimize
    return {"objective_value": mse}



# Define the parameters to optimize
parameters = [
    {"name": "n_estimators", "type": "range", "bounds": [10, 200], "value_type": "int"},
    {"name": "max_depth", "type": "range", "bounds": [5, 50], "value_type": "int"},
    {"name": "min_samples_split", "type": "range", "bounds": [2, 20], "value_type": "int"},
    {"name": "min_samples_leaf", "type": "range", "bounds": [1, 10], "value_type": "int"}
]

# Run the optimization
best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=evaluation_function,
    objective_name='objective_value',
    minimize=True,
    total_trials=30
)

[INFO 09-25 19:55:52] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='n_estimators', parameter_type=INT, range=[10, 200]), RangeParameter(name='max_depth', parameter_type=INT, range=[5, 50]), RangeParameter(name='min_samples_split', parameter_type=INT, range=[2, 20]), RangeParameter(name='min_samples_leaf', parameter_type=INT, range=[1, 10])], parameter_constraints=[]).
[INFO 09-25 19:55:52] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there is at least one ordered parameter and there are no unordered categorical parameters.
[INFO 09-25 19:55:52] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=4 num_trials=None use_batch_trials=False
[INFO 09-25 19:55:52] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=8
[INFO 09-25 19:55:52] ax.modelbridge.dispatch_utils: num_co

Now lets check our parameters and objective value. As you can see, the MSE score went down by a bit.

In [20]:
print("Best Parameters:", best_parameters)
print("Original Score:", mse)
print("Hyperparameter Tuned Score", values[0])

Best Parameters: {'n_estimators': 44, 'max_depth': 44, 'min_samples_split': 20, 'min_samples_leaf': 2}
Original Score: 0.0038889048983534588
Hyperparameter Tuned Score {'objective_value': 0.0020091875679146024}


## Conclusion

In this tutorial, we used Bayesian Optimization with the Ax library to efficiently find the minimum of a synthetic function, optimize within a dataset, and perform hyperparameter tuning on another model. Ax facilitated the process of defining the experiment, running the optimization, and analyzing the results. 

### Further Reading

- [Ax Documentation](https://ax.dev/)
- [More on Bayesian Optimization](https://arxiv.org/abs/1807.02811)
